#### 3. 랜덤 포레스트 (Random Forest)

- 배깅과 부스팅보다 더 많은 무작위성을 주어 약한 학습기들을 생성한 후 이를 선형결합하여 최종 학습기를 만드는 방법
- 수천 개의 변수를 변수 제거 없이 모델링하므로 정확도 측면에서 좋은 성과를 보임
- 이론적 설명이나 최종 결과에 대한 해석이 어렵다는 단점이 있지만 예측력이 매우 높은 것으로 알려져 있음
- 입력변수가 많은 경우 배깅과 부스팅과 비슷하거나 좋은 예측력을 보임

<br>

<b>분류의 경우</b>
> sklearn.ensemble.RandomForestClassifier(...)

<b>회귀의 경우</b>
> sklearn.ensemble.RandomForestRegressor(...)

- n_estimators : RandomForest에서 나무의 수 (int, default = 100)<br>
- criterion : 분할의 품질을 측정하는 방법 ({“gini”, “entropy”}, default=”gini”)<br>
- max_depth : 나무의 최대 깊이. (int, default = None)<br>
&emsp; &emsp;&emsp; &emsp;&emsp; None이면 모든 잎이 순수하거나 잎에 min_sample_split 미만의 샘플만 존재할 때까지 노드 확장<br>
- min_samples_split : 내부 노드를 분할하는 데 필요한 최소 샘플 수 (int or float, default=2)<br>
- min_samples_leaf : 리프 노드에 있어야하는 최소 샘플 수 (int or float, default=1)<br>
- max_leaf_nodes : 리프 노드의 최대 개수 (int, default = None)<br>
&emsp;&emsp;&emsp;&emsp; &emsp; &emsp; &emsp; None이면 리프 노드 수에 제한이 없음<br>
- bootstrap : 나무를 만들 때 부트 스트랩 샘플이 사용되는지 여부 (bool, default=True)<br>
&emsp;&emsp;&emsp;&emsp;&emsp; False이면 전체 데이터 세트가 각 트리를 작성하는 데 사용<br>
- oob_score : 정확도 측정을 위해 out-of-bag 샘플을 사용할지 여부 (bool, default=False)<br>

In [1]:
import pandas as pd 
df = pd.read_csv('C:/adp/data/kc_house_data.csv')
df = df.drop(['id','date'], axis=1)

X = df.drop('price',axis=1)
y = df['price']

X = pd.get_dummies(data = X, columns=['waterfront'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=2021)

In [3]:
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(n_estimators=50)

reg = reg.fit(X_train, y_train)

Y_pred = reg.predict(X_test)
print("train 정확도 : ", reg.score(X_train, y_train))

print("test 정확도 : ", reg.score(X_test, y_test))

train 정확도 :  0.9629911554884562
test 정확도 :  0.7593180652589903


변수 중요도

In [5]:
## 변수의 중요도 확인
import numpy as np 
importances = pd.DataFrame(np.mean([tree.feature_importances_ for tree in reg.estimators_], axis=0))
feature_importances = pd.concat([pd.DataFrame(X.columns), importances], axis=1)
feature_importances.columns=["col_name","feature_importance"]
feature_importances = feature_importances.sort_values(by="feature_importance",ascending=False).reset_index(drop=True)
feature_importances

,col_name,feature_importance
0,grade,0.356635
1,sqft_living,0.262476
2,yr_built,0.102469
3,sqft_living15,0.063564
4,sqft_lot15,0.046366
5,sqft_lot,0.035858
6,sqft_above,0.027561
7,bathrooms,0.022134
8,view,0.021527
9,sqft_basement,0.013505


Out of Bag 샘플을 활용한 성능 측정

In [6]:
reg = RandomForestRegressor(n_estimators=100, 
                             min_samples_split=5, 
                             oob_score=True)
reg = reg.fit(X, y)

print(f"oob score : {reg.oob_score_}")

oob score : 0.7543559166964672
